# Introduction

The aim of our project is to find out whether the product created from two designated molecules is stable. Indeed, our vision is to ask a user to draw two molecules of his choice. Via the "reaction_utils" template, a product is created. To find out whether it was stable and whether the reaction was spontaneous, we also chose to calculate free enthalpies.

Free enthalpy, or Gibbs free energy, ΔG is a thermodynamic function that determines whether a chemical reaction can occur spontaneously at constant temperature and pressure. It is calculated by the formula ΔG=ΔH-TΔS where H is enthalpy [J], T is the temperature [K] and S is the entropy [J/K]. A reaction is spontaneous if the variation in free enthalpy is negative. In our case,  If the enthalpy of the sum of the reactants is greater than that of the product, then the latter is stable. This concept is therefore essential for predicting chemical equilibrium and optimizing the conditions of chemical reactions, whether industrial or biological. Free enthalpy thus enables us to better understand the direction and feasibility of chemical transformations.


# Materials and Methods

To reach our goal, which is to determine if the reaction of two given reactive is spontaneous or not at 0K, the problem was devided in multiple steps:
First, a drawing interface was created to enable the users to draw their reactants.
Then, the reactants were analyzed to determine what possible product they could yield using reaction templates. 
Finally, the energy of all molecules both reactants and product were calculated and compaired to conclude on the spontaneous character of the reaction at 0K.

All this steps can be summaryze with this chart : 

Molecule ( Smiles “CCOC…”) → Convert into 3D (xyz) → Found the energy using XTB and Morfeus
A CHANGER

## 1. Creating a drawing interface

First, a drawing interface inspired by the materials provided in lecture 12 was created using streamlit and ketcher. The interface was designed for the user to draw the two reagents,enableling them to see their drawing as well as the corresponding smiles, 3D visualization and energy of the molecules. ALso, after chosen the type of wanted reaction, the product was also illustrated with the same features. Finally, we made sure that the full extent of the reaction was shown, and that if the reaction was favourable, we could see it.

One of the main functions of this part was therefore to obtain the molecule drawn and display its three functionalities. 







In [ ]:

def draw_and_process(title, session_key):
    """Draw a molecule and calculate its energy."""
    st.subheader(f"{title}")
    # interface
    smiles = st_ketcher(st.session_state.get(session_key, ""), key=f"{session_key}_ketcher", height=400)
    
    if smiles:
        # smiles
        st.session_state[session_key] = smiles
        with st.expander(f"SMILES"):
            st.code(smiles)
        
        # 3D 
        molblock = generate_3D(smiles)
        if molblock:
            with st.expander("3D Visualization"):
                visualize_3D(molblock)
            
            try:

                # energy 
                energy, elements, coords = calculate_energy_with_rdkit(smiles)
                
                # register the optimized geometry
                os.makedirs("xyz_files", exist_ok=True)
                xyz_path = os.path.abspath(f"xyz_files/{session_key}_optimized.xyz")
                write_xyz_file(elements, coords, xyz_path)
                
                st.session_state[f"{session_key}_energy"] = energy
                with st.expander("Energy"):
                    st.success(f"Energy (RDKit): {energy:.6f} Hartree")
                
            except Exception as e:
                st.error(f"Energy calculation failed: {str(e)}")
                import traceback
                st.code(traceback.format_exc())
    
    return smiles

One of the main challenges of this part was to integrate the product into the screen interface. Indeed, linking internal calculation functions to interface-related functions was a challenge. We had to make sure we were dealing with the same language, and many conversions had to be made to smarts, smiles or molblock.

Secondly, we chose to create a site that was easy for the reader to navigate. Streamlit was our choice. Nevertheless, when we open the site, the page “jumps” to the Molecule 2 section. We haven't found a solution to this problem. One way of solving it would have been to put the two interfaces for Molecule 1 and 2 next to each other, but then we'd lose the sibebar for choosing specific atoms such as Br, O or S. 



## 2. Generating a Product 

a toi maria bebou

## 3. Calculation of the molecule's energy 

Furthermore, the energy of each molecule was required. To obtain that information the initial idea was to convert the drawn molecule into a xyz file in order to optimize its geometry and calculate the energy of the molecule. The latter process presented many difficulties: 

# 3. Results and Discussion